In [1]:
import pandas as pd

import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, inspect

from sqlalchemy.exc import IntegrityError


from datetime import datetime
import math

In [2]:
# read in the master counties data, create a DataFrame, create new 'county_state' column
path = './PResources/Inflation_Rate.csv'
Inflation_RateDF = pd.read_csv(path)

Inflation_RateDF.head()

,Country,Category,DateTime,Value,Frequency,HistoricalDataSymbol,LastUpdate
0,United States,Inflation Rate,1914-12-31T00:00:00,1.0,Monthly,CPI YOY,2015-02-26T15:56:00
1,United States,Inflation Rate,1915-01-31T00:00:00,1.0,Monthly,CPI YOY,2015-02-26T15:56:00
2,United States,Inflation Rate,1915-02-28T00:00:00,1.0,Monthly,CPI YOY,2015-02-26T15:56:00
3,United States,Inflation Rate,1915-03-31T00:00:00,0.0,Monthly,CPI YOY,2015-02-26T15:56:00
4,United States,Inflation Rate,1915-04-30T00:00:00,2.0,Monthly,CPI YOY,2015-02-26T15:56:00


In [3]:
keep_col = ['Category','DateTime','Value']
Inflation_RateDF = Inflation_RateDF[keep_col].drop_duplicates(keep='first')


Inflation_RateDF

,Category,DateTime,Value
0,Inflation Rate,1914-12-31T00:00:00,1.0
1,Inflation Rate,1915-01-31T00:00:00,1.0
2,Inflation Rate,1915-02-28T00:00:00,1.0
3,Inflation Rate,1915-03-31T00:00:00,0.0
4,Inflation Rate,1915-04-30T00:00:00,2.0
...,...,...,...
1279,Inflation Rate,2021-07-31T00:00:00,5.4
1280,Inflation Rate,2021-08-31T00:00:00,5.3
1281,Inflation Rate,2021-09-30T00:00:00,5.4
1282,Inflation Rate,2021-10-31T00:00:00,6.2


In [4]:
# convert date column into date format
Inflation_RateDF['DateTime'] = pd.to_datetime(Inflation_RateDF['DateTime'])

Inflation_RateDF

,Category,DateTime,Value
0,Inflation Rate,1914-12-31,1.0
1,Inflation Rate,1915-01-31,1.0
2,Inflation Rate,1915-02-28,1.0
3,Inflation Rate,1915-03-31,0.0
4,Inflation Rate,1915-04-30,2.0
...,...,...,...
1279,Inflation Rate,2021-07-31,5.4
1280,Inflation Rate,2021-08-31,5.3
1281,Inflation Rate,2021-09-30,5.4
1282,Inflation Rate,2021-10-31,6.2


In [5]:
# filter rows on the basis of date
Inflation_RateDF = Inflation_RateDF[((Inflation_RateDF['DateTime'] > '01/01/2017') & (Inflation_RateDF['DateTime'] <= '12/31/2021'))]


Inflation_RateDF

,Category,DateTime,Value
1225,Inflation Rate,2017-01-31,2.5
1226,Inflation Rate,2017-02-28,2.7
1227,Inflation Rate,2017-03-31,2.4
1228,Inflation Rate,2017-04-30,2.2
1229,Inflation Rate,2017-05-31,1.9
1230,Inflation Rate,2017-06-30,1.6
1231,Inflation Rate,2017-07-31,1.7
1232,Inflation Rate,2017-08-31,1.9
1233,Inflation Rate,2017-09-30,2.2
1234,Inflation Rate,2017-10-31,2.0


In [6]:
Inflation_RateDF['Difference'] =  Inflation_RateDF['Value'] - Inflation_RateDF['Value'].shift(1)
Inflation_RateDF = Inflation_RateDF.fillna(0)

Inflation_RateDF

C:\Users\14074\AppData\Local\Temp/ipykernel_5404/3793323879.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Inflation_RateDF['Difference'] =  Inflation_RateDF['Value'] - Inflation_RateDF['Value'].shift(1)


,Category,DateTime,Value,Difference
1225,Inflation Rate,2017-01-31,2.5,0.0
1226,Inflation Rate,2017-02-28,2.7,0.2
1227,Inflation Rate,2017-03-31,2.4,-0.3
1228,Inflation Rate,2017-04-30,2.2,-0.2
1229,Inflation Rate,2017-05-31,1.9,-0.3
1230,Inflation Rate,2017-06-30,1.6,-0.3
1231,Inflation Rate,2017-07-31,1.7,0.1
1232,Inflation Rate,2017-08-31,1.9,0.2
1233,Inflation Rate,2017-09-30,2.2,0.3
1234,Inflation Rate,2017-10-31,2.0,-0.2


In [7]:
#Inflation_RateDF ='./Output/inflation_rate.csv'
Inflation_RateDF.to_csv("Inflater.csv", index=False)
#type(Inflation_RateDF)